# Install Necessary packages


In [10]:
pip install -U datasets

  Using cached datasets-2.15.0-py3-none-any.whl.metadata (20 kB)
Using cached datasets-2.15.0-py3-none-any.whl (521 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.12.0
    Uninstalling datasets-2.12.0:
      Successfully uninstalled datasets-2.12.0
Note: you may need to restart the kernel to use updated packages.


# Importing Libraries

In [3]:
import torch
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
gc.collect()
torch.cuda.empty_cache()

In [5]:
# model_name = "gpt2-medium"
model_name = "gpt2"
dataset_name = "HuggingFaceH4/ultrachat_200k"

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [6]:
train_dataset = load_dataset(dataset_name, split='train_gen')
train_dataset.to_pandas()

C:\Users\shash\anaconda3\lib\site-packages\datasets\table.py:1424: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


,prompt,prompt_id,messages
0,How can I fix errors in my grammar when writin...,c033f4b800f390a3e9bb4266d22242c90852a1ef8dee6f...,[{'content': 'How can I fix errors in my gramm...
1,Create a comprehensive guide for writing a pro...,532aec37e5778500677847da77e15ab2d05f40dba9c84d...,[{'content': 'Create a comprehensive guide for...
2,How do people in Hinduism practice daily devot...,173bc31d726a5e8b5e3381a159d98d168a5a94e963449b...,[{'content': 'How do people in Hinduism practi...
3,How does the concept of torsion fields help ex...,74ea19b8ebeeab77180987254f06007fec14ab1d9d8012...,[{'content': 'How does the concept of torsion ...
4,Numerical Modeling of Multi-Species Transport ...,fedf09781f1405da3b2dbc0b5f0a572275b58d2c02e200...,[{'content': 'Numerical Modeling of Multi-Spec...
...,...,...,...
256027,How does the lymphatic system contribute to im...,829f5887f74c1ccdfad6cdf3d96c615fd2eda96c1681b8...,[{'content': 'How does the lymphatic system co...
256028,"Most weekends this semester, you’d find me in ...",fdcd85f6bc6c99ec6f0dd7904103f087e45f2638bea865...,"[{'content': 'Most weekends this semester, you..."
256029,Who were the top contenders in the 1984 U.S. O...,3e98da789057180b68e0bde2e7825dc1f50d6cf80a8956...,[{'content': 'Who were the top contenders in t...
256030,Build a C++ program that calculates the sum of...,8d631e52ca4a32022a7f3d5d2395fe4582211788d78271...,[{'content': 'Build a C++ program that calcula...


In [7]:
test_dataset = load_dataset(dataset_name, split='test_gen')
test_dataset.to_pandas()

,prompt,prompt_id,messages
0,"This story begins with an end. In March 1991, ...",5ee2fbb48ef35593b81444d7aec405bb4f152abbe80f7b...,[{'content': 'This story begins with an end. I...
1,Explain how the invention and widespread use o...,fc6aae406cd26c79db4d35dd32bcbd8ee0f1493a0096b5...,[{'content': 'Explain how the invention and wi...
2,Read the passage below and answer the question...,44a13514d9cd363d85479ff25e5837c60c5f90815428c2...,[{'content': 'Read the passage below and answe...
3,Explain the influence of culture on attitudes ...,c0c7f2a08bd4dc84bc527d774b1fe411eefa7bcdb847b5...,[{'content': 'Explain the influence of culture...
4,Can you provide data on the employment rates i...,b26cb026578e891c3ccd0cf075da6cffaa05df05412aa0...,[{'content': 'Can you provide data on the empl...
...,...,...,...
28299,How have the TIMI trials contributed to our un...,42cf5424e1f8a3ddf7670dd9273620cfeb42c9d2c3d746...,[{'content': 'How have the TIMI trials contrib...
28300,Write step-by-step instructions for making a h...,da5a99e17e7be7a4ee51e8f54709a0347a6e82afa8cf1a...,[{'content': 'Write step-by-step instructions ...
28301,"Using Unity, create a puzzle game that allows ...",597c69c1b58ba7d049ef405ca37045a8c3b7de6898e7e4...,"[{'content': 'Using Unity, create a puzzle gam..."
28302,Please research and find a charity organizatio...,dfe500129711acbbd609fe4b7f1f19414cecd6e041c403...,[{'content': 'Please research and find a chari...


In [12]:
def prepare_datasets(data):
    prompt = ""
    for i in range(len(data['messages']) - 1):
        if len(prompt) > 0:
            prompt = prompt + "\n" + f"""{data['messages'][i]['role']}: {data['messages'][i]['content']}"""
        else:
            prompt = f"""{data['messages'][i]['role']}: {data['messages'][i]['content']}"""
    data['query'] = prompt
    return data


def tokenize_datasets(dataset):
  tokenized_dataset = dataset.map(
      lambda example: tokenizer(
          example['query'],
          truncation=True,
          max_length=128,
          ),
      batched=True,
      remove_columns=['query']
  )
  return tokenized_dataset

In [15]:
train_dataset = train_dataset.map(
    prepare_datasets, remove_columns=['prompt', 'prompt_id', 'messages']		
)

test_dataset = test_dataset.map(
    prepare_datasets, remove_columns=['prompt', 'prompt_id', 'messages']		
)

In [16]:
train_dataset

Dataset({
    features: ['query'],
    num_rows: 256032
})

In [17]:
test_dataset

Dataset({
    features: ['query'],
    num_rows: 28304
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [19]:
train_dataset = tokenize_datasets(train_dataset)
test_dataset = tokenize_datasets(test_dataset)

In [20]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True), mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [21]:
batch_size = 4
training_args = TrainingArguments(
    output_dir="./models/chat_gpt2",
    gradient_accumulation_steps=batch_size,
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    save_strategy="no",
    save_total_limit=2,
    fp16=True,
    learning_rate=2e-05,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to=None
)


training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=4,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

In [23]:
trainer.train()

C:\Users\shash\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.174400
1000,2.832500
1500,2.726800
2000,2.659500
2500,2.623200
3000,2.579500
3500,2.557900
4000,2.521700
4500,2.525600
5000,2.507600


TrainOutput(global_step=48006, training_loss=2.385711920945261, metrics={'train_runtime': 26819.7887, 'train_samples_per_second': 28.639, 'train_steps_per_second': 1.79, 'total_flos': 5.0174341152768e+16, 'train_loss': 2.385711920945261, 'epoch': 3.0})

In [9]:
MODEL_PATH = "Sharathhebbar24/chat_gpt2"
HF_TOKEN = "<YOUR HF TOKEN>"

In [8]:
tokenizer.push_to_hub(
    MODEL_PATH,
    token=HF_TOKEN
)

C:\Users\shash\anaconda3\lib\site-packages\huggingface_hub\_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'merges.txt'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
C:\Users\shash\anaconda3\lib\site-packages\huggingface_hub\_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'special_tokens_map.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
C:\Users\shash\anaconda3\lib\site-packages\huggingface_hub\_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
C:\Users\shash\anaconda3\lib\site-packages\huggingface_hub\_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/Sharathhebbar24/chat_gpt2/commit/2d1f29a101a38d244f63c97c68c2872d3216bccf', commit_message='Upload tokenizer', commit_description='', oid='2d1f29a101a38d244f63c97c68c2872d3216bccf', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub(
    MODEL_PATH,
    token=HF_TOKEN
)